In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



[Stage 1:>                                                          (0 + 1) / 1]

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



In [4]:
cdf.columns

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### 각 반 학생수(class_std_cnt)가 학생수의 평균보다 많은 class의 data를 추출하시오

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']
반코드        학교이름   반 학생수       지역    학교형태       교육형태

In [11]:
# view 생성
cdf.createOrReplaceTempView('classV')

In [12]:
# where 절에서 평균 반 학생수를 구하고 그 값보다 많은 반을 추출
spark.sql('''
    select * from classV
    where class_std_cnt > avg(class_std_cnt) and class_std_cnt is not null
''').show()

AnalysisException: [INVALID_WHERE_CONDITION] The WHERE condition "((class_std_cnt > avg(class_std_cnt)) AND (class_std_cnt IS NOT NULL))" contains invalid expressions: avg(CAST(classv.class_std_cnt AS DOUBLE)).
Rewrite the query to avoid window functions, aggregate functions, and generator functions in the WHERE clause.; line 2 pos 4;
Project [class_cd#17, school#18, class_std_cnt#19, loc#20, school_type#21, teaching_type#22]
+- Filter ((cast(class_std_cnt#19 as double) > avg(cast(class_std_cnt#19 as double))) AND isnotnull(class_std_cnt#19))
   +- SubqueryAlias classv
      +- View (`classV`, [class_cd#17,school#18,class_std_cnt#19,loc#20,school_type#21,teaching_type#22])
         +- Relation [class_cd#17,school#18,class_std_cnt#19,loc#20,school_type#21,teaching_type#22] csv


## 에러발생 : where절에서는 집계함수 사용 불가
- select / having 절에서만 집계 함수 사용 가능
- spark.sql('''
            select * from classV
            where class_std_cnt > 계산한 평균값 and class_std_cnt is not null''').show()
   이렇게 쓸 수는 있다.

- 평균을 직접 구해서 평균을 쓰고 select하면 가능
- 근데 이러면 코드가 너무 구려지잖아
- 평균을 구하는 쿼리를 이용해서 평균값을 받고 해당 쿼리를 where 조건에 추가하는 걸로 해결할 수 있다.

- subquery 사용
  1. subquery 이용 평균 반 학생수를 전달하기
  2. where 절에서 반 학생수가 1에서 전달받은 값보다 큰 data 추출

In [14]:
# 1. 평균 구하는 subquery - 평균 반 학생수 select 하는 쿼리(집계함수는 select 절에만 사용가능)
spark.sql('''
    select avg(class_std_cnt) from classV
    where class_std_cnt is not null
''').show()

[Stage 4:>                                                          (0 + 1) / 1]

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [16]:
# 2. 평균 구하는 쿼리를 where절에서 사용하는 subquery로 구성 - 1의 query를 where에 비교 대상으로 사용(subquery 적용)
spark.sql('''
    select * from classV
    where class_std_cnt >= (select avg(class_std_cnt) from classV
                            where class_std_cnt is not null)
''').show()

[Stage 8:>                                                          (0 + 1) / 1]

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
|     ROP| DNQDD|           28|Suburban|     Public| Experimental|
|     XXJ| DNQDD|           27|Suburban|     Public|     Standard|
|     HCB| GJJHK|           22|Suburban|     Public|     Standard|
|     NOR| GJJHK|           27|Suburban|     Public| Experimental|
|     ZDT| GJJHK|           27|Suburban|     Public|     Standard|
|     ENO| GOKXL|           22|   Rural|     Public| Experimental|
|     TSA| GOKXL|           23|   Rural|     Public| Experimen

### df method 통해 subquery와 같은 결과를 도출해보자

In [17]:
# 1. 평균 구하는 식
cdf.select(avg('class_std_cnt')).show()

[Stage 9:>                                                          (0 + 1) / 1]

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [19]:
cdf.select(avg('class_std_cnt'))

DataFrame[avg(class_std_cnt): double]

In [18]:
# 2. 1에서 구한 평균값을 활용해서 평균보다 많은 학생 수 추출
# 아래 코드의 cdf.select(avg('class_std_cnt')) 연산식은 반환값이 **수치가 아니라 객체임** - 객체 id가 연산결과로 나옴
# where절을 진행할 수 없음
cdf.select('*')\
    .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')))\
    .show()

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [25]:
# 연산식의 평균값을 수치로 반환받아야함
# 값을 반환하는 action method: collect()
cdf.select(avg('class_std_cnt')).collect()   #출력값: [Row(avg(class_std_cnt)=21.828282828282827)]
type(cdf.select(avg('class_std_cnt')).collect()) #리스트래
cdf.select(avg('class_std_cnt')).collect()[0]  #그럼 값 가져오게 인덱싱하자 ==> 안되네
type(cdf.select(avg('class_std_cnt')).collect()[0]) #타입이 뭔데 안되지? ==? Row(집합)
cdf.select(avg('class_std_cnt')).collect()[0][0] #집합이니까 한번 더 접근하자
type(cdf.select(avg('class_std_cnt')).collect()[0][0]) #드디어 숫자가 나왔다.

cdf.select('*')\
    .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')).collect()[0][0])\
    .show()

[Row(avg(class_std_cnt)=21.828282828282827)]

list

Row(avg(class_std_cnt)=21.828282828282827)

pyspark.sql.types.Row

21.828282828282827

float

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
|     ROP| DNQDD|           28|Suburban|     Public| Experimental|
|     XXJ| DNQDD|           27|Suburban|     Public|     Standard|
|     HCB| GJJHK|           22|Suburban|     Public|     Standard|
|     NOR| GJJHK|           27|Suburban|     Public| Experimental|
|     ZDT| GJJHK|           27|Suburban|     Public|     Standard|
|     ENO| GOKXL|           22|   Rural|     Public| Experimental|
|     TSA| GOKXL|           23|   Rural|     Public| Experimen

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


- spark.sql.dataframe.DataFrame 객체도 RDD기반임
    - show() 출력만 진행함
    - collect() 사용가능: 객체 반환 가능
        - 객체 타입에 따라 index 사용이 가능 : 실제값을 추출할 수 있음
    - collect()사용한 연산식은 더이상 지연은 안됨 : action연산

복잡한 연산도 해보자

#### case 1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수를 구해보자
#### 단, 학교가 null인 데이터는 제외한다

In [44]:
# from절 subquery : inline view

cdf.createOrReplaceTempView('class')

spark.sql('''
    select min(std_cnt) 
    from (select school, sum(class_std_cnt) as std_cnt 
            from class 
            where school is not null
            group by school
            having count(school) >= 3 )
''').show(3)

+------------+
|min(std_cnt)|
+------------+
|        46.0|
+------------+



+------------+
|min(std_cnt)|
+------------+
|        46.0|
+------------+



#### case 1_1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수와 학교명을 추출하자
#### 단, 학교가 null인 데이터는 제외한다

### sql query

In [74]:
spark.sql(''' select school, std_tot
            from (select school, sum(class_std_cnt) as std_tot from class group by school having school is not null and count(school)>=3)
            where std_tot == (select min(std_tot) as min_stdtot
                              from(select school, sum(class_std_cnt) as std_tot 
                                   from class 
                                   group by school 
                                   having school is not null and count(school)>=3))  
    
    ''').show()

+------+-------+
|school|std_tot|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### df moethod 사용

+------+-------+
|school|std_cnt|
+------+-------+
| VHDHF|   51.0|
| LAYPA|   57.0|
| GOOBU|  158.0|
| UUUQX|   84.0|
| CIMBB|   74.0|
| UKPGS|  128.0|
| UAGPU|   87.0|
| CCAAW|  109.0|
| FBUMG|   46.0|
| ZOWMK|  117.0|
| ZMNYA|   69.0|
| QOQTS|  149.0|
| CUQAM|  107.0|
| OJOBU|   81.0|
| GOKXL|   64.0|
| GJJHK|  118.0|
| KZKKE|  111.0|
| DNQDD|  122.0|
| VKWQH|  100.0|
| IDGFP|   94.0|
+------+-------+
only showing top 20 rows



+------+-------+
|school|std_cnt|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### case2. 1.지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교별로 반의 학생수가 가장 작은 반의 학생수를 구하시오
### 위에서 구한 학생수중 가장 큰 값은 얼마인가?
1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

### sql query 활용

In [127]:
# 참고
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [126]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출


+--------+------+---+
|     loc|school|cnt|
+--------+------+---+
|   Rural| VHDHF|  3|
|   Urban| GOOBU|  6|
|Suburban| ZMNYA|  3|
|   Urban| ZOWMK|  4|
|    null|  null|  3|
|Suburban| UUUQX|  5|
|Suburban| DNQDD|  5|
|   Urban| CUQAM|  4|
|   Urban| IDGFP|  5|
|   Rural| GOKXL|  3|
|   Rural| KZKKE|  5|
|   Rural| VKWQH|  5|
|Suburban| CCAAW|  6|
|   Rural| LAYPA|  3|
|   Urban| QOQTS|  6|
|Suburban| UAGPU|  4|
|   Rural| FBUMG|  3|
|   Urban| VVTVA|  4|
|Suburban| UKPGS|  6|
|Suburban| GJJHK|  5|
|   Rural| OJOBU|  4|
|   Urban| CIMBB|  4|
+--------+------+---+



In [128]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [124]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출


+--------+------+---+-------+
|     loc|school|cnt|min_std|
+--------+------+---+-------+
|   Rural| FBUMG|  3|     14|
|   Rural| GOKXL|  3|     19|
|   Rural| KZKKE|  5|     20|
|   Rural| LAYPA|  3|     17|
|   Rural| OJOBU|  4|     17|
|   Rural| VHDHF|  3|     15|
|   Rural| VKWQH|  5|     18|
|Suburban| CCAAW|  6|     15|
|Suburban| DNQDD|  5|     20|
|Suburban| GJJHK|  5|     21|
|Suburban| UAGPU|  4|     21|
|Suburban| UKPGS|  6|     18|
|Suburban| UUUQX|  5|     15|
|Suburban| ZMNYA|  3|     22|
|   Urban| CIMBB|  4|     17|
|   Urban| CUQAM|  4|     24|
|   Urban| GOOBU|  6|     24|
|   Urban| IDGFP|  5|     17|
|   Urban| QOQTS|  6|     22|
|   Urban| VVTVA|  4|     25|
|   Urban| ZOWMK|  4|     27|
+--------+------+---+-------+



In [125]:
# 3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다

+------------+
|max(min_std)|
+------------+
|          27|
+------------+



### df의 메서드 활용

In [80]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [91]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출

        

+--------+------+------+
|     loc|school|cnt_cd|
+--------+------+------+
|   Rural| VHDHF|     3|
|   Urban| GOOBU|     6|
|Suburban| ZMNYA|     3|
|   Urban| ZOWMK|     4|
|Suburban| UUUQX|     5|
|Suburban| DNQDD|     5|
|   Urban| CUQAM|     4|
|   Urban| IDGFP|     5|
|   Rural| GOKXL|     3|
|   Rural| KZKKE|     5|
|   Rural| VKWQH|     5|
|Suburban| CCAAW|     6|
|   Rural| LAYPA|     3|
|   Urban| QOQTS|     6|
|Suburban| UAGPU|     4|
|   Rural| FBUMG|     3|
|   Urban| VVTVA|     4|
|Suburban| UKPGS|     6|
|Suburban| GJJHK|     5|
|   Rural| OJOBU|     4|
|   Urban| CIMBB|     4|
+--------+------+------+



In [108]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [92]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
# 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수


+--------+------+------+-----------+
|     loc|school|cnt_cd|min_std_cnt|
+--------+------+------+-----------+
|   Rural| FBUMG|     3|         14|
|   Rural| GOKXL|     3|         19|
|   Rural| KZKKE|     5|         20|
|   Rural| LAYPA|     3|         17|
|   Rural| OJOBU|     4|         17|
|   Rural| VHDHF|     3|         15|
|   Rural| VKWQH|     5|         18|
|Suburban| CCAAW|     6|         15|
|Suburban| DNQDD|     5|         20|
|Suburban| GJJHK|     5|         21|
|Suburban| UAGPU|     4|         21|
|Suburban| UKPGS|     6|         18|
|Suburban| UUUQX|     5|         15|
|Suburban| ZMNYA|     3|         22|
|   Urban| CIMBB|     4|         17|
|   Urban| CUQAM|     4|         24|
|   Urban| GOOBU|     6|         24|
|   Urban| IDGFP|     5|         17|
|   Urban| QOQTS|     6|         22|
|   Urban| VVTVA|     4|         25|
|   Urban| ZOWMK|     4|         27|
+--------+------+------+-----------+



In [107]:
# 3. 2번에서 추출된 학생수들 중 가장 많은 수 추출
# # 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수에서 가장 큰 수 
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다


+----------------+
|max(min_std_cnt)|
+----------------+
|              27|
+----------------+



### case3. 지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교에서 학교별 가장 작은 학생수들을 추출 그 중에서 가장 큰 수를 구하시오(27)
### 구한 수보다 학생수가 더 많은 반과 학생수를 cdf 전체 데이터에서 추출하시오

#### sql 쿼리

In [130]:

spark.sql('select count(*) from class').show()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은  전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+

+--------+
|count(1)|
+--------+
|     102|
+--------+



#### df 메서드

In [110]:

cdf.count()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은 전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+



102

#### case 4. 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교를 추출하시오

In [ ]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수


+------------------+
|avg(class_std_cnt)|
+------------------+
|20.928571428571427|
+------------------+



+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     98D| IDGFP|           21|Urban| Non-public| Experimental|
|     1VD| KFZMY|           27|Urban| Non-public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [138]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수

+---+
|avg|
+---+
| 21|
+---+



21

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     98D| IDGFP|           21|Urban| Non-public| Experimental|
|     1VD| KFZMY|           27|Urban| Non-public| Experimental|
+--------+------+-------------+-----+-----------+-------------+

